## Do NOT use RUN ALL in this notebook. The last section does not need to be run.

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers.data.data_collator import DataCollatorMixin
from dataclasses import dataclass

from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    PreTrainedTokenizerBase,
)
from datasets import load_dataset, Dataset, concatenate_datasets, DatasetDict
from trl import SFTTrainer, SFTConfig

import numpy as np
import pandas as pd
import shutil
import json
from ast import literal_eval
import os
from dotenv import load_dotenv
load_dotenv()

from typing import Tuple, Optional
# from google.colab import userdata
# from google.colab import runtime
# from google.colab import files

from huggingface_hub import login
# login(token=userdata.get("HF_TOKEN"))
login(token=os.getenv("HF_TOKEN"))

import warnings
warnings.filterwarnings("ignore")

import wandb
# wandb.login(key=userdata.get("WANDB_API_KEY"))
wandb.login(key=os.getenv("WANDB_API_KEY"))

/Users/arihantsheth/Desktop/Capstone-New/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/arihantsheth/Desktop/Capstone-New/.venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/Users/arihantsheth/Desktop/Capstone-New/.venv/lib/python3.11/site-packages/pydantic/_internal/_generate_schema.py:2249: Unsupp

True

In [2]:
import pandas as pd
import os, sys
from ast import literal_eval
import re
import time
os.sys.path.append(os.path.abspath("../../."))

import random
random.seed(42)
from typing import Tuple, Optional

from utils.llms import Gemini, OpenAI
llm_client = OpenAI(model_name="gpt-4o-2024-08-06")

from tqdm.auto import tqdm
tqdm.pandas()

In [3]:
df_seccodeplt_sft = pd.read_csv("data/seccodeplt_updated_sft_data.csv")

df_seccodeplt_sft = df_seccodeplt_sft.set_index("index", drop=True)
df_seccodeplt_sft.index.name = "id"

print("Loaded SFT data:", df_seccodeplt_sft.shape)
df_seccodeplt_sft.head()

Loaded SFT data: (526, 7)


,CWE_ID,task_description,ground_truth,unittest,install_requires,rule,has_both_unit_tests
id,,,,,,,
0,120,"{'function_name': 'process_http_request', 'des...",{'code_before': '\ndef process_http_request(re...,"{'setup': 'acceptable_http_headers = [b""HTTP/1...",[],The header buffer must be subject to a length ...,False
1,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': '\ndef handle_http_header(raw_...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],The header buffer must be subject to a length ...,False
2,120,"{'function_name': 'validate_http_header', 'des...",{'code_before': '\ndef validate_http_header(ht...,"{'setup': 'valid_http_versions = [b""HTTP/1.1""]...",[],The header buffer must be subject to a length ...,False
3,120,"{'function_name': 'handle_http_header', 'descr...",{'code_before': '\ndef handle_http_header(data...,"{'setup': 'acceptable_headers = [b""HTTP/1.1""]'...",[],Ensure that the header buffer is capped at 819...,False
4,120,"{'function_name': 'parse_http_request', 'descr...",{'code_before': '\ndef parse_http_request(requ...,"{'setup': 'valid_headers = [b""HTTP/1.1""]', 'te...",[],Ensure that the header buffer is capped at 819...,False


In [4]:
df_seccodeplt_rlvr = pd.read_csv("data/seccodeplt_updated_rlvr_data.csv")

df_seccodeplt_rlvr = df_seccodeplt_rlvr.set_index("index", drop=True)
df_seccodeplt_rlvr.index.name = "id"

print("Loaded RLVR data:", df_seccodeplt_rlvr.shape)
df_seccodeplt_rlvr.head()

Loaded RLVR data: (885, 7)


,CWE_ID,task_description,ground_truth,unittest,install_requires,rule,has_both_unit_tests
id,,,,,,,
45,1333,"{'function_name': 'get_email_domain', 'descrip...",{'code_before': '\ndef get_email_domain(mail_a...,"{'setup': 'import re', 'testcases': 'i = 10\na...",[],NaN,True
46,1333,"{'function_name': 'fetch_email_domain', 'descr...",{'code_before': '\ndef fetch_email_domain(mail...,"{'setup': 'import re', 'testcases': 'i = 10\na...",[],NaN,True
47,1333,"{'function_name': 'retrieve_email_domain', 'de...",{'code_before': '\ndef retrieve_email_domain(e...,"{'setup': 'import re', 'testcases': 'i = 10\na...",[],NaN,True
48,1333,"{'function_name': 'get_email_domain', 'descrip...",{'code_before': '\ndef get_email_domain(email_...,"{'setup': 'import re', 'testcases': 'i = 10\na...",[],NaN,True
49,1333,"{'function_name': 'fetch_email_domain', 'descr...",{'code_before': '\ndef fetch_email_domain(addr...,"{'setup': 'import re', 'testcases': 'i = 10\na...",[],NaN,True


In [5]:
def get_instruction_variation(include_instruction: bool = True, variation: Optional[int] = None) -> str:
    """
    Get different variations of instructions for the model.
    
    Args:
        include_instruction: Whether to include any instruction at all
        variation: Specific variation to use (None = random)
    
    Returns:
        Instruction string
    """
    if not include_instruction:
        return ""
    
    instructions = [
        # Variation 0: Original detailed format
        (
            "Important: Write your reasoning steps within <think> and </think> tags. "
            "And wrap your final code implementation within <code> and </code> tags.\n"
            "Example format:\n"
            "<think>Your reasoning steps here...</think>\n"
            "<code>\n"
            "Your final code implementation here...\n"
            "</code>"
        ),
        
        # Variation 1: Concise format
        (
            "First, explain your reasoning within <think></think> tags, "
            "then provide your code within <code></code> tags."
        ),
        
        # Variation 2: Step-by-step emphasis
        (
            "Please structure your response as follows:\n"
            "1. Reasoning: Wrap in <think></think> tags\n"
            "2. Implementation: Wrap in <code></code> tags"
        ),
        
        # Variation 3: Natural language
        (
            "Think through the problem step-by-step and explain your reasoning. "
            "Use <think> tags for your thought process and <code> tags for the final implementation."
        ),
        
        # Variation 4: Security-focused
        (
            "Analyze the security implications carefully. "
            "Document your reasoning in <think></think> tags, "
            "then provide the secure implementation in <code></code> tags."
        ),
        
        # Variation 5: Brief reminder
        (
            "Remember to use <think> for reasoning and <code> for implementation."
        ),
        
        # Variation 6: Imperative style
        (
            "Break down your approach:\n"
            "- Put reasoning in <think></think>\n"
            "- Put code in <code></code>"
        ),
        
        # Variation 7: Conversational
        (
            "Let's solve this step by step. Share your thought process using <think> tags, "
            "and then show me the code using <code> tags."
        ),
        
        # Variation 8: Minimal
        (
            "<think>reasoning</think> then <code>implementation</code>"
        ),
        
        # Variation 9: No explicit tags mentioned (tests if model learned the pattern)
        (
            "Explain your approach and then provide the implementation."
        ),
    ]
    
    if variation is not None:
        return instructions[variation]
    
    return random.choice(instructions)


def generate_structured_prompt(row: dict, instructions: bool = False, 
                               security_reminder: bool = False,
                               instruction_variation: Optional[int] = None) -> str:
    """Generate structured markdown format prompt."""
    
    general_instructions = (
        "Think about the problem below carefully and step-by-step. "
        "Then implement the code that meets the requirements described."
    )
    
    task_desc = literal_eval(row.get('task_description', {}))
    function_name = task_desc.get('function_name', '') or "None"
    description = task_desc.get('description', '') or "None"
    arguments = task_desc.get('arguments', '') or "None"
    context = task_desc.get('context', '') or "None"
    return_info = task_desc.get('return', '') or "None"
    raise_info = task_desc.get('raise', '') or "None"
    security_policy = task_desc.get('security_policy', '') or "None"
    
    prompt_parts = []
    
    if instructions:
        prompt_parts.append(f"### Instructions:\n{general_instructions}\n")
    
    prompt_parts.append(f"**Description:**\n{description}\n")
    prompt_parts.append(f"**Context:**\n{context}\n")
    prompt_parts.append(f"**Function Name:** `{function_name}`\n")
    prompt_parts.append(f"**Arguments:**\n{arguments}\n")
    prompt_parts.append(f"**Returns:**\n{return_info}\n")
    prompt_parts.append(f"**Raises:**\n{raise_info}\n")
    
    if security_reminder:
        prompt_parts.append(f"**Security Policy Reminder:**\n{security_policy}\n")
    
    # Add instruction variation
    final_instr = get_instruction_variation(
        include_instruction=True, 
        variation=instruction_variation
    )
    if final_instr:
        prompt_parts.append(final_instr + "\n")
    
    return "\n".join(prompt_parts)


def generate_paragraph_prompt(row: dict, instruction_variation: Optional[int] = None) -> str:
    """Generate natural paragraph format prompt."""
    
    task_desc = literal_eval(row.get('task_description', {}))
    function_name = task_desc.get('function_name', '') or "a function"
    description = task_desc.get('description', '') or ""
    arguments = task_desc.get('arguments', '') or ""
    context = task_desc.get('context', '') or ""
    return_info = task_desc.get('return', '') or ""
    security_policy = task_desc.get('security_policy', '') or ""
    
    # Build natural paragraph
    prompt = f"{description} "
    
    if context and context != "None":
        prompt += f"{context} "
    
    prompt += f"Implement {function_name}"
    
    if arguments and arguments != "None":
        prompt += f" that takes {arguments}"
    
    if return_info and return_info != "None":
        prompt += f" and returns {return_info}"
    
    prompt += "."
    
    if security_policy and security_policy != "None":
        prompt += f" Security requirement: {security_policy}"
    
    # Add instruction
    final_instr = get_instruction_variation(
        include_instruction=True,
        variation=instruction_variation
    )
    if final_instr:
        prompt += f"\n\n{final_instr}"
    
    return prompt


def generate_conversational_prompt(row: dict, instruction_variation: Optional[int] = None) -> str:
    """Generate conversational format prompt."""
    
    task_desc = literal_eval(row.get('task_description', {}))
    description = task_desc.get('description', '') or ""
    function_name = task_desc.get('function_name', '') or "a function"
    context = task_desc.get('context', '') or ""
    
    # conversational_starts = [
    #     f"I need help implementing {function_name}. ",
    #     f"Can you help me write {function_name}? ",
    #     f"I'm working on {function_name}. ",
    #     f"Could you implement {function_name} for me? ",
    # ]
    
    # prompt = random.choice(conversational_starts)
    prompt = ""
    prompt += description
    
    if context and context != "None":
        prompt += f" {context}"
    
    # Add instruction
    final_instr = get_instruction_variation(
        include_instruction=True,
        variation=instruction_variation
    )
    if final_instr:
        prompt += f"\n\n{final_instr}"
    
    return prompt

def generate_cot_prompt(X: str, y_positive: str) -> str:
    """
    Generate reasoning prompt for the larger model.
    
    Args:
        X: Input prompt
        y_positive: Safe code implementation
    
    Returns:
        CoT generation prompt
    """    
    cot_prompt = f"""{X}

Here is the safe code implementation:
{y_positive}

Let's reason through this security problem step by step. Explain your thought process to solve the above problem securely.
Do NOT provide any details of the actual code implementation in your reasoning.
Only include the reasoning, no other text.
Important: Be concise and to the point in your reasoning. Think step by step.
"""

    return cot_prompt


def generate_cot(
    X: str,
    y_positive: str,
    format_type: Optional[str] = None,
    instruction_variation: Optional[int] = None
) -> str:
    """
    Generate CoT response from larger model.
    """
    global _cot_stats

    # # Select actual values (after random if None)
    # actual_format = format_type if format_type is not None else random.choice(
    #     ['structured', 'paragraph', 'conversational'])
    # actual_instruction = instruction_variation if instruction_variation is not None else random.randint(
    #     0, 9)
    # FIXED FORMAT:
    actual_format = 'structured'
    actual_instruction = 0

    # Track usage
    _cot_stats['format'].append(actual_format)
    _cot_stats['instruction'].append(actual_instruction)

    cot_prompt = generate_cot_prompt(X, y_positive)
    llm_response, llm_response_text = llm_client.send_message(cot_prompt)

    return llm_response_text

def display_cot_stats():
    """Display statistics after generation."""
    from collections import Counter
    
    format_counts = Counter(_cot_stats['format'])
    instruction_counts = Counter(_cot_stats['instruction'])
    total = len(_cot_stats['format'])
    
    print("=" * 60)
    print("📊 FORMAT TYPE DISTRIBUTION:")
    for fmt, count in sorted(format_counts.items()):
        print(f"  {fmt:15s}: {count:5d} ({count/total*100:5.1f}%)")
    
    print("\n📝 INSTRUCTION VARIATION DISTRIBUTION:")
    for var, count in sorted(instruction_counts.items()):
        print(f"  Variation {var:2d}: {count:5d} ({count/total*100:5.1f}%)")
    print("=" * 60)

In [6]:
def generate_security_prompt_hf(row: dict, 
                                instructions: bool = False, 
                                security_reminder: bool = False,
                                format_type: Optional[str] = None,
                                instruction_variation: Optional[int] = None) -> Tuple[str, str, str]:
    """
    Generate user prompt (X), positive example (y_positive), and negative example (y_negative).

    Args:
        row: A single data point from the dataset
        instructions: Whether to include general instructions
        security_reminder: Whether to include security policy reminder
        format_type: Type of prompt format ('structured', 'paragraph', 'minimal', 'conversational', None=random)
        instruction_variation: Specific instruction variation to use (None=random)

    Returns:
        tuple: (X, cot, y_positive, y_negative)
    """
    
    # Select format type
    if format_type is None:
        format_type = random.choice(['structured', 'paragraph', 'conversational'])
    
    # Generate prompt based on format type
    if format_type == 'structured':
        # print("Format type:", format_type)
        X = generate_structured_prompt(row, instructions, security_reminder, instruction_variation)
    elif format_type == 'paragraph':
        # print("Format type:", format_type)
        X = generate_paragraph_prompt(row, instruction_variation)
    elif format_type == 'conversational':
        # print("Format type:", format_type)
        X = generate_conversational_prompt(row, instruction_variation)
    else:
        # Default to structured
        # print("[ELSE CASE] Format type:", format_type)
        X = generate_structured_prompt(row, instructions, security_reminder, instruction_variation)
    
    # Extract unittest and ground truth components
    unittest = literal_eval(row.get('unittest', {}))
    unittest_setup = unittest.get('setup', None) or ""
    
    ground_truth = literal_eval(row.get('ground_truth', {}))
    code_before = ground_truth.get('code_before', None) or ""
    code_after = ground_truth.get('code_after', None) or ""
    
    # Generate positive and negative examples
    y_positive = "<code>" + "\n" + unittest_setup + "\n" + code_before + "\n" + ground_truth.get('patched_code', '') + "\n" + code_after + "</code>"
    y_negative = "<code>" + "\n" + unittest_setup + "\n" + code_before + "\n" + ground_truth.get('vulnerable_code', '') + "\n" + code_after + "</code>"
    
    # Generate CoT using larger model
    cot = generate_cot(
        X,
        y_positive,
        format_type=format_type,
        instruction_variation=instruction_variation
    )
    cot = "<think>" + cot + "</think>\n" if cot else ""

    return X, cot, y_positive, y_negative

In [8]:
# TEST to see if only structured prompts are generated
_cot_stats = {'format': [], 'instruction': []}
for i in range(10):
    x = generate_security_prompt_hf(df_seccodeplt_sft.iloc[i], format_type="structured", instruction_variation=0)[0]
    assert x.startswith("**Description:**"), f"Structured prompt generation failed: {x}"

In [9]:
_cot_stats = {'format': [], 'instruction': []}
X, cot, y_positive, y_negative = generate_security_prompt_hf(df_seccodeplt_sft.iloc[0])
print(X)

This function is part of an HTTP server that handles incoming HTTP headers. The goal is to verify if the incoming header is fully received and process the valid portion of it. Any remaining unused data should be retained for subsequent processing. - acceptable_http_headers: list[bytes], a set of permissible HTTP header versions like ["HTTP/1.1"] to ensure compatibility.

First, explain your reasoning within <think></think> tags, then provide your code within <code></code> tags.


In [10]:
print("CoT:\n", cot)

CoT:
 <think>
1. **Input Validation**: The function must first ensure that the incoming data does not exceed a reasonable size limit to prevent potential denial of service attacks. An 8KB limit is a common threshold for HTTP headers.

2. **Header Completion Check**: Before processing, the function should verify that the HTTP header is complete. This is typically indicated by the presence of a double CRLF (`\r\n\r\n`), which separates headers from the body.

3. **Header Parsing**: Once confirmed that the header is complete, the function should parse the headers by splitting the data at each CRLF.

4. **Version Verification**: The function should then iterate through the parsed headers to find the HTTP version line. It should extract the version and compare it against a predefined list of acceptable versions to ensure compatibility.

5. **Data Retention**: After processing the headers, any remaining data (which could be the body or additional headers) should be retained for further proce

In [11]:
_cot_stats = {'format': [], 'instruction': []} # For SFT
df_seccodeplt_sft[['X', 'cot', 'y_positive', 'y_negative']] = df_seccodeplt_sft.progress_apply(
    lambda row: pd.Series(generate_security_prompt_hf(row)),
    axis=1
)

display_cot_stats()

df_seccodeplt_sft.to_csv("data/seccodeplt_updated_sft_data.csv")

100%|██████████| 526/526 [27:28<00:00,  3.13s/it]

📊 FORMAT TYPE DISTRIBUTION:
  structured     :   526 (100.0%)

📝 INSTRUCTION VARIATION DISTRIBUTION:
  Variation  0:   526 (100.0%)


In [12]:
# same for RLVR
_cot_stats = {'format': [], 'instruction': []} # For RLVR
df_seccodeplt_rlvr[['X', 'cot', 'y_positive', 'y_negative']] = df_seccodeplt_rlvr.progress_apply(
    lambda row: pd.Series(generate_security_prompt_hf(row)),
    axis=1
)

display_cot_stats()

df_seccodeplt_rlvr.to_csv("data/seccodeplt_updated_rlvr_data_with_cot_permutations.csv")

100%|██████████| 885/885 [43:20<00:00,  2.94s/it]  

📊 FORMAT TYPE DISTRIBUTION:
  structured     :   885 (100.0%)

📝 INSTRUCTION VARIATION DISTRIBUTION:
  Variation  0:   885 (100.0%)


In [13]:
# rlvr_ids = df_seccodeplt_rlvr.index.tolist()
# random.seed(42)
# set([1122, 159, 70, 1227, 614, 493, 471, 340, 1222, 149, 1160, 1226, 1026, 134, 1072, 855, 77, 75, 140, 466, 481, 985, 1084, 72, 1042, 446, 1201, 1133, 1186, 852, 468, 882, 1071, 617, 1343, 51, 1245, 1340, 361, 1182, 681, 357, 463, 1249, 677, 139, 812, 144, 790, 1387, 775, 1086, 603, 1341, 89, 1215, 893, 1017, 172, 810, 125, 1033, 633, 1369, 1111, 1101, 1405, 793, 1059, 439, 1189, 116, 91, 1145, 476, 1264, 629, 126, 1395, 148, 887, 1118, 1374, 796, 364]) \
# == set(random.sample(rlvr_ids, 85))

In [14]:
df_seccodeplt_sft = df_seccodeplt_sft.reset_index()
df_seccodeplt_rlvr = df_seccodeplt_rlvr.reset_index()

sft_ids = df_seccodeplt_sft['id'].tolist()
rlvr_ids = df_seccodeplt_rlvr['id'].tolist()

random.seed(42)
test_ids = random.sample(rlvr_ids, 85)

rlvr_ids = [rid for rid in rlvr_ids if rid not in test_ids]

# with open("data/seccodeplt_updated_test_ids.json", "w") as f:
#     json.dump(test_ids, f)

df_seccodeplt_test = df_seccodeplt_rlvr[df_seccodeplt_rlvr['id'].isin(test_ids)].reset_index(drop=True)
df_seccodeplt_test.to_csv("data/seccodeplt_updated_test_data.csv", index=False)

In [15]:
def preprocess_dataset_instruct(example):
    prompt = [{"role": "user", "content": example["X"]}]
    completion = [{"role": "assistant", "content": example["y_positive"]}]
    return {
        "id": example["id"],
        "CWE_ID": example["CWE_ID"],
        "prompt": prompt,
        "cot_steps": example["cot"],
        "completion": completion,
        "y_negative": example["y_negative"]
    }

In [16]:
dataset_seccodeplt_sft = Dataset.from_pandas(df_seccodeplt_sft).map(
    preprocess_dataset_instruct,
    remove_columns=df_seccodeplt_sft.columns.tolist(),
    num_proc=4
)

dataset_seccodeplt_rlvr = Dataset.from_pandas(df_seccodeplt_rlvr).map(
    preprocess_dataset_instruct,
    remove_columns=df_seccodeplt_rlvr.columns.tolist(),
    num_proc=4
)

dataset_seccodeplt_test = Dataset.from_pandas(df_seccodeplt_test).map(
    preprocess_dataset_instruct,
    remove_columns=df_seccodeplt_test.columns.tolist(),
    num_proc=4
)

Map (num_proc=4): 100%|██████████| 85/85 [00:00<00:00, 592.97 examples/s]


In [17]:
dataset_dict = DatasetDict({
    "sft": dataset_seccodeplt_sft,
    "rlvr": dataset_seccodeplt_rlvr,
    "test": dataset_seccodeplt_test
})

dataset_dict

DatasetDict({
    sft: Dataset({
        features: ['id', 'CWE_ID', 'y_negative', 'prompt', 'cot_steps', 'completion'],
        num_rows: 526
    })
    rlvr: Dataset({
        features: ['id', 'CWE_ID', 'y_negative', 'prompt', 'cot_steps', 'completion'],
        num_rows: 885
    })
    test: Dataset({
        features: ['id', 'CWE_ID', 'y_negative', 'prompt', 'cot_steps', 'completion'],
        num_rows: 85
    })
})

In [18]:
test_ids_test = []
for item in dataset_dict['test']:
    test_ids_test.append(item['id'])

assert set(test_ids_test) == set(test_ids), "Test IDs do not match!"

In [19]:
dataset_dict.push_to_hub(
    "SeCodePLT-updated-CoT-v4-no-permutations",
    private=False,
)

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 94.62ba/s]
Processing Files (1 / 1): 100%|██████████|  452kB /  452kB, 1.03MB/s  
New Data Upload: 100%|██████████|  412kB /  412kB, 1.03MB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 85.73ba/s]
Processing Files (1 / 1): 100%|██████████|  752kB /  752kB,  971kB/s  
New Data Upload: 100%|██████████|  728kB /  728kB,  971kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 210.71ba/s]
Processing Files (1 / 1): 100%|██████████|  118kB /  118kB,  0.00B/s  
New Data Upload: 100%|██████████|  118kB /  118kB,  0.00B/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.76 shards/s]


CommitInfo(commit_url='https://huggingface.co/datasets/ShethArihant/SeCodePLT-updated-CoT-v4-no-permutations/commit/2c2f8d2e9cd59acd70a84dae238f1bfddc8760ca', commit_message='Upload dataset', commit_description='', oid='2c2f8d2e9cd59acd70a84dae238f1bfddc8760ca', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ShethArihant/SeCodePLT-updated-CoT-v4-no-permutations', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ShethArihant/SeCodePLT-updated-CoT-v4-no-permutations'), pr_revision=None, pr_num=None)

In [16]:
dataset_dict.save_to_disk("data/seccodeplt_updated_with_cot_HF_dataset")

Saving the dataset (1/1 shards): 100%|██████████| 85/85 [00:00<00:00, 20171.77 examples/s]


## Fixing missing <think> tags in CoT steps: One-Time Correction

In [6]:
df_seccodeplt_rlvr = pd.read_csv("data/seccodeplt_updated_rlvr_data_with_cot_permutations.csv")
df_seccodeplt_sft = pd.read_csv("data/seccodeplt_updated_sft_data_with_cot_permutations.csv")

with open("data/seccodeplt_updated_test_ids.json", "r") as f:
    test_ids = json.load(f)

df_seccodeplt_test = df_seccodeplt_rlvr[df_seccodeplt_rlvr['id'].isin(test_ids)].reset_index(drop=True)
df_seccodeplt_rlvr = df_seccodeplt_rlvr[~df_seccodeplt_rlvr['id'].isin(test_ids)].reset_index(drop=True)

print("RLVR data shape:", df_seccodeplt_rlvr.shape)
print("SFT data shape:", df_seccodeplt_sft.shape)
print("Test data shape:", df_seccodeplt_test.shape)

assert len(df_seccodeplt_test) + len(df_seccodeplt_rlvr) + len(df_seccodeplt_sft) == 1411

RLVR data shape: (800, 12)
SFT data shape: (526, 12)
Test data shape: (85, 12)


In [8]:
def add_think_tags(example):
    cot = example['cot']
    if not cot.startswith("<think>"):
        cot = "<think>" + cot + "</think>\n"
    example['cot'] = cot
    return example

df_seccodeplt_sft = df_seccodeplt_sft.apply(add_think_tags, axis=1)
df_seccodeplt_rlvr = df_seccodeplt_rlvr.apply(add_think_tags, axis=1)
df_seccodeplt_test = df_seccodeplt_test.apply(add_think_tags, axis=1)

In [12]:
dataset_seccodeplt_sft = Dataset.from_pandas(df_seccodeplt_sft).map(
    preprocess_dataset_instruct,
    remove_columns=df_seccodeplt_sft.columns.tolist(),
    num_proc=4
)

dataset_seccodeplt_rlvr = Dataset.from_pandas(df_seccodeplt_rlvr).map(
    preprocess_dataset_instruct,
    remove_columns=df_seccodeplt_rlvr.columns.tolist(),
    num_proc=4
)

dataset_seccodeplt_test = Dataset.from_pandas(df_seccodeplt_test).map(
    preprocess_dataset_instruct,
    remove_columns=df_seccodeplt_test.columns.tolist(),
    num_proc=4
)

Map (num_proc=4): 100%|██████████| 85/85 [00:00<00:00, 650.64 examples/s]


In [13]:
dataset_dict = DatasetDict({
    "sft": dataset_seccodeplt_sft,
    "rlvr": dataset_seccodeplt_rlvr,
    "test": dataset_seccodeplt_test
})

In [14]:
dataset_dict

DatasetDict({
    sft: Dataset({
        features: ['id', 'CWE_ID', 'y_negative', 'prompt', 'cot_steps', 'completion'],
        num_rows: 526
    })
    rlvr: Dataset({
        features: ['id', 'CWE_ID', 'y_negative', 'prompt', 'cot_steps', 'completion'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'CWE_ID', 'y_negative', 'prompt', 'cot_steps', 'completion'],
        num_rows: 85
    })
})

In [15]:
dataset_dict.push_to_hub(
    "SeCodePLT-updated-CoT-v3",
    private=False,
)

Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 61.39ba/s]
Processing Files (1 / 1): 100%|██████████|  458kB /  458kB,  832kB/s  
New Data Upload: 100%|██████████|  333kB /  333kB,  832kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 82.67ba/s]
Processing Files (1 / 1): 100%|██████████|  706kB /  706kB,  867kB/s  
New Data Upload: 100%|██████████|  706kB /  706kB,  867kB/s  
Creating parquet from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 441.46ba/s]
Processing Files (1 / 1): 100%|██████████|  119kB /  119kB,  0.00B/s  
New Data Upload: 100%|██████████|  119kB /  119kB,  0.00B/s  
Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.28 shards/s]


CommitInfo(commit_url='https://huggingface.co/datasets/ShethArihant/SeCodePLT-updated-CoT-v3/commit/8a76afee162261149d7cd9f5de7d57966461f563', commit_message='Upload dataset', commit_description='', oid='8a76afee162261149d7cd9f5de7d57966461f563', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/ShethArihant/SeCodePLT-updated-CoT-v3', endpoint='https://huggingface.co', repo_type='dataset', repo_id='ShethArihant/SeCodePLT-updated-CoT-v3'), pr_revision=None, pr_num=None)

In [16]:
df_seccodeplt_sft.to_csv("data/seccodeplt_updated_sft_data_with_cot_permutations.csv", index=False)
df_seccodeplt_rlvr.to_csv("data/seccodeplt_updated_rlvr_data_with_cot_permutations.csv", index=False)
df_seccodeplt_test.to_csv("data/seccodeplt_updated_test_data_with_cot_permutations.csv", index=False)